"title": "My Title!",
"authors": [
  { "name": "Author1" },
  { "name": "Author2" }
]

%%bash
# install packages
pip install -r ../requirements.txt

In [2]:
# Third party packages
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import matplotlib.pyplot as plt
from io import StringIO

from os.path import abspath
import sys
sys.path.insert(0, abspath('..'))

from query_request import *
from query_def import *
import credentials
# from sqlalchemy import create_engine

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth=None


In [3]:
%%time
from rdfizer.semantify import semantify
# semantify('../configs/config.ini')

CPU times: user 75.5 ms, sys: 1.7 ms, total: 77.2 ms
Wall time: 72.8 ms


In [ ]:
# create rdfmts
from DeTrusty import run_query
from DeTrusty.Molecule.MTCreation import create_rdfmts
from DeTrusty.Molecule.MTManager import ConfigFile

endpoints = dictfrmjson('../configs/endpoints_public.json')
print(endpoints)
# log_rdf = create_rdfmts(endpoints, '../configs/rdfmts.json')

log_rdf = create_rdfmts(endpoints, '../configs/rdfmts_public.json')
# if already rdfmts has been created 
#config = ConfigFile('../configs/rdfmts_backup.json')


Total time taken by dictfrmjson function: 6.222724914550781e-05
{'https://labs.tib.eu/sdm/icews_endpoint/sparql/': {'mappings': ['../mappings/icews/icews.ttl', '../mappings/country/country_codes.ttl']}, 'https://labs.tib.eu/sdm/lei_endpoint/sparql/': {'mappings': ['../mappings/lei/lei.ttl']}, 'https://labs.tib.eu/sdm/worldbank_endpoint/sparql': {'mappings': ['../mappings/worldbank/worldbank_0.ttl', '../mappings/worldbank/worldbank_7.ttl']}}


2022-11-28 16:23:36,023 - rdfmts - INFO - Finding inter-links between: https://labs.tib.eu/sdm/icews_endpoint/sparql/ and https://labs.tib.eu/sdm/lei_endpoint/sparql/
2022-11-28 16:23:36,026 - rdfmts - INFO - ==============================//=========//===============================
2022-11-28 16:23:36,035 - rdfmts - INFO - Finding inter-links between: https://labs.tib.eu/sdm/icews_endpoint/sparql/ and https://labs.tib.eu/sdm/worldbank_endpoint/sparql
2022-11-28 16:23:36,039 - rdfmts - INFO - ==============================//=========//===============================
2022-11-28 16:23:36,050 - rdfmts - INFO - Finding inter-links between: https://labs.tib.eu/sdm/lei_endpoint/sparql/ and https://labs.tib.eu/sdm/icews_endpoint/sparql/
2022-11-28 16:23:36,053 - rdfmts - INFO - ==============================//=========//===============================
2022-11-28 16:23:36,142 - rdfmts - INFO - Finding inter-links between: https://labs.tib.eu/sdm/lei_endpoint/sparql/ and https://labs.tib.eu/sdm

In [ ]:
# cmemc_request = SPARQLRequest(client_url_imp, client_id_imp, client_secret_imp , 'oauth')

skynet_request = SPARQLRequest(skynet_endpoint, skynet_user, skynet_pass, 'basic')

# fdquery_request = SPARQLRequest("http://localhost:5002/sparql", is_fdq=True) for docker

# otherwise

run_query(query_1_fdq, config=config, join_stars_locally=False)


In [ ]:
# Executing test query on Skynet server
print(query_0)
skynet_request.execute(query_0)
buf = StringIO(skynet_request.response.content.decode('utf-8'), newline='\r\n')          
pd.read_csv(buf, encoding='utf-8')

In [ ]:
print(query_1_fdq_desc)
print(query_1_fdq)
print('########################### EXAMPLE ##########################\n{}'.format(query_1_fdq_ex))
fdquery.post_query(query_1_fdq_ex)
fdquery.to_save('../../query_results/', filename='query_1_fdq_ex')

In [ ]:
query_1_fdq

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port
        ),)

df = pd.read_csv('../../query_results/query_1_fdq_ex.csv', encoding='utf-8')
df.rename(columns=lambda x: x.replace('.value', ''), inplace=True)
try:
    df.to_sql('query_1_fdq_ex', con=engine, if_exists='fail', index=False)
except Exception as e:
    print('Exeception: {}',e)

In [ ]:
from sqlalchemy import create_engine

engine = create_engine(
        "mysql+pymysql://{user}:{pw}@{hostname}:{portno}/{db}".format(
            hostname=host, db=dbname, user=uname, pw=pwd, portno=port
        ),)

query_1_fdq_ex_sql = """SELECT country_code, year, AVG(value) as carbon_emission, COUNT(disaster) AS no_of_disasters
FROM `query_1_fdq_ex`
WHERE year=year_dis
GROUP By country_code, year
Order By  year ASC
"""

df = pd.read_sql(query_1_fdq_ex_sql, con=engine)

df.to_csv('../../query_results/query_1_fdq_ex_sql.csv', encoding='utf-8', index=False)

In [ ]:
def carbon_emission_group(carbon_emission):
    if carbon_emission/1000000 < 0.25:
        return '0.00-0.25'
    elif carbon_emission/1000000 >= 0.25 and carbon_emission/1000000 < 0.5:
        return '0.25-0.5'
    elif carbon_emission/1000000 >= 0.5 and carbon_emission/1000000 < 0.75:
        return '0.50-0.75'
    else:
        return '> 0.75'
    

df = pd.read_csv('../../query_results/query_1_fdq_ex_sql.csv', encoding='utf-8')
df['carbon_emission(1e6)'] = df['carbon_emission'].apply(carbon_emission_group)

df.head(10)

In [ ]:

fig = px.scatter_geo(df, locations="country_code", color="carbon_emission(1e6)",
                     hover_data=['carbon_emission'], size="no_of_disasters",
                     animation_frame="year",
                     projection="equirectangular", scope='europe')
fig.update_layout(title_text='Carbon emission and the number of disaster events for various countries across years',\
                  title_font_color="red", title_font_family="Times New Roman", title_font_size=25)
fig.write_html('../../query_results/query_1_fdq_ex_sql_new.html')
fig.show()